## Web scraping theses.fr for the PhD dataset 

Alexandre Huet

1. Install the needed libraries

In [94]:
#!pip install requests
#!pip install BeautifulSoup4
import requests
from bs4 import BeautifulSoup

2. Get the url of all pages 

url of page 1 = https://theses.fr/resultats?q=*&page=1&nb=10&tri=pertinence&domaine=theses

In [95]:
for i in range(1,11): 
    start_url = "https://theses.fr/resultats?q=*&page="
    page = i
    end_url ="&nb=10&tri=pertinence&domaine=theses"

    url = start_url+str(page)+end_url
    print(url)

https://theses.fr/resultats?q=*&page=1&nb=10&tri=pertinence&domaine=theses
https://theses.fr/resultats?q=*&page=2&nb=10&tri=pertinence&domaine=theses
https://theses.fr/resultats?q=*&page=3&nb=10&tri=pertinence&domaine=theses
https://theses.fr/resultats?q=*&page=4&nb=10&tri=pertinence&domaine=theses
https://theses.fr/resultats?q=*&page=5&nb=10&tri=pertinence&domaine=theses
https://theses.fr/resultats?q=*&page=6&nb=10&tri=pertinence&domaine=theses
https://theses.fr/resultats?q=*&page=7&nb=10&tri=pertinence&domaine=theses
https://theses.fr/resultats?q=*&page=8&nb=10&tri=pertinence&domaine=theses
https://theses.fr/resultats?q=*&page=9&nb=10&tri=pertinence&domaine=theses
https://theses.fr/resultats?q=*&page=10&nb=10&tri=pertinence&domaine=theses


3. Request access to those url 

Output: <Response[200]> means that the response is ok

In [96]:
for i in range(1,11): 
    start_url = "https://theses.fr/resultats?q=*&page="
    page = i
    end_url ="&nb=10&tri=pertinence&domaine=theses"

    # Create the url
    url = start_url+str(page)+end_url
    print(url)

    # Request access for each url 
    r = requests.get(url)

    # Check the response
    print(r)

https://theses.fr/resultats?q=*&page=1&nb=10&tri=pertinence&domaine=theses
<Response [200]>
https://theses.fr/resultats?q=*&page=2&nb=10&tri=pertinence&domaine=theses
<Response [200]>
https://theses.fr/resultats?q=*&page=3&nb=10&tri=pertinence&domaine=theses
<Response [200]>
https://theses.fr/resultats?q=*&page=4&nb=10&tri=pertinence&domaine=theses
<Response [200]>
https://theses.fr/resultats?q=*&page=5&nb=10&tri=pertinence&domaine=theses
<Response [200]>
https://theses.fr/resultats?q=*&page=6&nb=10&tri=pertinence&domaine=theses
<Response [200]>
https://theses.fr/resultats?q=*&page=7&nb=10&tri=pertinence&domaine=theses
<Response [200]>
https://theses.fr/resultats?q=*&page=8&nb=10&tri=pertinence&domaine=theses
<Response [200]>
https://theses.fr/resultats?q=*&page=9&nb=10&tri=pertinence&domaine=theses
<Response [200]>
https://theses.fr/resultats?q=*&page=10&nb=10&tri=pertinence&domaine=theses
<Response [200]>


(Close the request, to not get blocked afterwards)

In [97]:
r.close()

4. Make the beautiful soup from the r.text 

In [98]:
soup = BeautifulSoup(r.text,"html.parser")
#soup

Go on the website to the relevant part that you want in the dataset and get from xpath to the "find" code 

In [99]:
soup.find_all("div")

[<div id="__nuxt"></div>]

In [100]:
# Find the element containing the href using the div and a tag structure
# # Adjust the tags and classes based on the actual HTML structure
target_element = soup.select_one('div#id__nuxt div div div div div div a') 
# Extract the href attribute
if target_element:     
    href_value = target_element.get('href')     
    full_url = "https://theses.fr" + href_value     
    print(full_url)
else: print("Element not found")

Element not found


In [101]:
#pip install selenium

## Web scraping from https://theses.fr/2022UPASB052

In [102]:
url = "https://theses.fr/2022UPASB052"

# Request access for each url 
r_ex = requests.get(url)

# Check the response
print(r_ex)

r_ex.close()

<Response [200]>


In [103]:
soup_ex = BeautifulSoup(r_ex.text,"html.parser")
#soup_ex

In [104]:
#pip install webdriver_manager

We use selenium here just to extract the urls. Selenium is however more time consuming, so after this we will extract all elements with Beautiful soup, since this is more time efficient.

In [ ]:
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Set up the Chrome WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Define the base URL for the year 2022 with pagination
base_url = 'https://theses.fr/resultats?filtres=%255Bdatefin%25253D%2525222022%252522~datedebut%25253D%2525222022%252522~Statut%25253D%252522soutenue%252522%255D&q=*&page='

# Set total pages and batch size
total_pages = 1  # Total number of pages to scrape (adjust accordingly)
batch_size = 1     # Define how many pages to process per batch

# Regular expression to match the desired URL format
pattern = re.compile(r'https://theses\.fr/[A-Za-z0-9]+')

# Function to scrape URLs from a page
def scrape_urls_from_page(page_number):
    full_url = f"{base_url}{page_number}&nb=500&tri=dateDesc&domaine=theses"
    print(f"Processing page {page_number}: {full_url}")
    hrefs = []

    try:
        # Navigate to the page URL
        driver.get(full_url)
        
        # Wait for the content to load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'first-half')))
        
        # Extract all hrefs from anchor tags
        anchor_elements = driver.find_elements(By.TAG_NAME, 'a')
        for anchor in anchor_elements:
            href = anchor.get_attribute('href')
            if href and pattern.match(href):  # Filter valid thesis URLs
                hrefs.append(href)

        time.sleep(1)  # Short delay to prevent server overload

    except TimeoutException:
        print(f"Timed out waiting for the page to load on page {page_number}. Skipping.")

    return hrefs

# Function to batch process scraping
def process_in_batches(total_pages, batch_size):
    all_hrefs = []
    for batch_start in range(1, total_pages + 1, batch_size):
        batch_end = min(batch_start + batch_size - 1, total_pages)
        print(f"Processing batch: Pages {batch_start} to {batch_end}")

        # Collect hrefs from the batch of pages
        for page_num in range(batch_start, batch_end + 1):
            hrefs = scrape_urls_from_page(page_num)
            all_hrefs.extend(hrefs)

        # Save the hrefs after each batch
        with open(f'thesis_links_batch_{batch_start}_to_{batch_end}.txt', 'w') as f:
            for href in all_hrefs:
                f.write(href + '\n')

        print(f"Batch {batch_start} to {batch_end} processed. Total collected URLs: {len(all_hrefs)}")

    return all_hrefs

# Start scraping in batches
all_hrefs = process_in_batches(total_pages, batch_size)

# Close the WebDriver after scraping
driver.quit()

Processing batch: Pages 1 to 1
Processing page 1: https://theses.fr/resultats?filtres=%255Bdatefin%25253D%2525222022%252522~datedebut%25253D%2525222022%252522~Statut%25253D%252522soutenue%252522%255D&q=*&page=1&nb=10&tri=dateDesc&domaine=theses
Batch 1 to 1 processed. Total collected URLs: 50


In [106]:
#all_hrefs

Filter the hrefs

In [107]:
import re

# Regular expression pattern to match the desired URLs
pattern = r'^https://theses\.fr/(?=.*[a-zA-Z])[a-zA-Z0-9]+$'

# Filter the hrefs list to keep only valid thesis URLs
filtered_hrefs = [
    href for href in all_hrefs 
    if re.match(pattern, href) 
    and not href.split('/')[-1].isdigit()  # Check that the last part is not purely numeric
    and not href.endswith('X')  # Check that the URL does not end with 'x'
    and not href.endswith('apropos')
]

#filtered_hrefs


In [108]:
print(f"Total number of collected URLs: {len(filtered_hrefs)}")

Total number of collected URLs: 10


Still need to add a for loop to itterate over multiple pages

Collect the data from the link

In [109]:
import pandas as pd

In [110]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Function to extract and print data from each URL
def extract_thesis_data(url, retries=3):
    attempt = 0
    while attempt < retries:
        try:
            # Fetch the page content with a timeout
            response = requests.get(url, timeout=10)
            response.raise_for_status()  # Raises an HTTPError for bad responses

            # Parse the HTML content with BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')

            # Getting the title of each
            title = soup.select_one('title')
            title = title.text.strip() if title else "Not found"

            author = soup.select_one('div:nth-of-type(1) > div > div > div:nth-of-type(2) > table > tbody > tr:nth-of-type(1) > td:nth-of-type(2)')
            author = author.text.strip() if author else "Not found"

            direction_people = soup.select_one('div:nth-of-type(1) > div > div > div:nth-of-type(2) > table > tbody > tr:nth-of-type(2) > td:nth-of-type(2)')
            direction_people = direction_people.text.strip() if direction_people else "Not found"

            thesis_type = soup.select_one('div:nth-of-type(1) > div > div > div:nth-of-type(2) > table > tbody > tr:nth-of-type(3) > td:nth-of-type(2) > span')
            thesis_type = thesis_type.text.strip() if thesis_type else "Not found"

            discipline = soup.select_one('div:nth-of-type(1) > div > div > div:nth-of-type(2) > table > tbody > tr:nth-of-type(4) > td:nth-of-type(2)')
            discipline = discipline.text.strip() if discipline else "Not found"

            soutenance = soup.select_one('div:nth-of-type(1) > div > div > div:nth-of-type(2) > table > tbody > tr:nth-of-type(5) > td:nth-of-type(2)')
            soutenance = soutenance.text.strip() if soutenance else "Not found"

            ecoledoctorale = soup.select_one('div:nth-of-type(1) > div > div > div:nth-of-type(2) > table > tbody > tr:nth-of-type(7) > td:nth-of-type(2) > strong > a')
            ecoledoctorale = ecoledoctorale.get_text(strip=True) if ecoledoctorale else "NaN"

            etablissement = soup.select_one('div:nth-of-type(1) > div > div > div:nth-of-type(2) > table > tbody > tr:nth-of-type(6) > td:nth-of-type(2)')
            etablissement = etablissement.text.strip() if etablissement else "Not found"

            partenaire = soup.select_one('div:nth-of-type(1) > div > div > div:nth-of-type(2) > table > tbody > tr:nth-of-type(8) > td:nth-of-type(2)')
            partenaire = partenaire.text.strip() if partenaire else "Not found"

            # Initialize jury variables
            jury_found = False
            president = "NaN"
            examinateurs = "NaN"
            rapporteurs = "NaN"

            # Loop through the row numbers for the jury selectors
            for i in range(7, 13):  
                jury_selector = soup.select_one(f"div:nth-of-type(1) > div > div > main > div > div.thesis-main-wrapper > div > div > div.thesis-info-access-wrapper > div.thesis-info-wrapper > table > tbody > tr:nth-of-type({i}) > td:nth-of-type(1)")

                # Check if the jury_selector is found and contains 'jury'
                if jury_selector and 'jury' in jury_selector.get_text(strip=True).lower():
                    jury_found = True
                    break  # Stop the loop once the jury is found

            if jury_found:
                # Adjust the row numbers for Président, Examinateurs, and Rapporteurs based on i
                president_row = i
                examinateurs_row = president_row + 1
                rapporteurs_row = president_row + 2

                # Extract 'Président'
                president_element = soup.select_one(f"div:nth-of-type(1) > div > div > div:nth-of-type(2) > table > tbody > tr:nth-of-type({president_row}) > td:nth-of-type(2) > a, div:nth-of-type(1) > div > div > div:nth-of-type(2) > table > tbody > tr:nth-of-type({president_row}) > td:nth-of-type(2) > span")
                if president_element:
                    president = president_element.get_text(strip=True)

                # Extract 'Examinateurs'
                examinateurs_element = soup.select_one(f"div:nth-of-type(1) > div > div > div:nth-of-type(2) > table > tbody > tr:nth-of-type({examinateurs_row}) > td:nth-of-type(2)")
                if examinateurs_element:
                    a_exam = examinateurs_element.find_all(["a", "span"])  # Find all <a> and <span> elements
                    examinateurs = " ".join(a.get_text(strip=True) for a in a_exam) if a_exam else "NaN"

                # Extract 'Rapporteurs'
                rapporteurs_element = soup.select_one(f"div:nth-of-type(1) > div > div > div:nth-of-type(2) > table > tbody > tr:nth-of-type({rapporteurs_row}) > td:nth-of-type(2)")
                if rapporteurs_element:
                    a_rapport = rapporteurs_element.find_all(["a", "span"])  # Find all <a> and <span> elements
                    rapporteurs = " ".join(a.get_text(strip=True) for a in a_rapport) if a_rapport else "NaN"

            # Return all variables as dictionary
            return {
                'Title': title, 
                'Author': author, 
                'Direction': direction_people, 
                'Type': thesis_type, 
                'Discipline': discipline, 
                'Date de soutenance': soutenance, 
                'Établissement': etablissement, 
                'Laboratoire': partenaire, 
                'Ecole doctorale': ecoledoctorale,  # Added Ecole doctorale
                'Président': president, 
                'Examinateurs': examinateurs,
                'Rapporteurs': rapporteurs
            }

        except requests.exceptions.Timeout:
            attempt += 1
            print(f"Timeout error for {url}, attempt {attempt}/{retries}. Retrying...")
            time.sleep(1)  # Wait 1 second before retrying

        except requests.exceptions.RequestException as e:
            print(f"Error fetching {url}: {e}")
            return {}

    print(f"Failed to fetch {url} after {retries} attempts.")
    return {}

# Creates the dataset
def process_extraction_in_batches(filtered_hrefs, batch_size):
    all_theses = []  # List to store all the extracted thesis data

    for batch_start in range(0, len(filtered_hrefs), batch_size):
        batch_end = min(batch_start + batch_size, len(filtered_hrefs))
        batch_hrefs = filtered_hrefs[batch_start:batch_end]
        print(f"Processing batch {batch_start} to {batch_end}")

        for url in batch_hrefs:
            print(f"Extracting data from: {url}")
            extracted_data = extract_thesis_data(url)
            if extracted_data:
                all_theses.append(extracted_data)

    PhD = pd.DataFrame(all_theses)
    
    # Return the DataFrame
    return PhD

# Run the extraction and save the returned DataFrame
PhD = process_extraction_in_batches(filtered_hrefs, batch_size=50)

Processing batch 0 to 10
Extracting data from: https://theses.fr/2022ESMA0023
Extracting data from: https://theses.fr/2022UBFCK091
Extracting data from: https://theses.fr/2022DUNK0642
Extracting data from: https://theses.fr/2022DUNK0654
Extracting data from: https://theses.fr/2022UMOND045
Extracting data from: https://theses.fr/2022ULILA029
Extracting data from: https://theses.fr/2022UPASJ025
Extracting data from: https://theses.fr/2022IPPAX130
Extracting data from: https://theses.fr/2022TOU30280
Extracting data from: https://theses.fr/2022LORR0347


In [111]:
PhD.head()

,Title,Author,Direction,Type,Discipline,Date de soutenance,Établissement,Laboratoire,Ecole doctorale,Président,Examinateurs,Rapporteurs
0,Efficacité énergétique des phases de conceptio...,Issam Ghabri,"Ladjel Bellatreche, Sadok Ben Yahia",Thèse de doctorat,Informatique et applications,Soutenance le 29/12/2022,"Chasseneuil-du-Poitou, Ecole nationale supérie...",Laboratoire : Laboratoire d'Informatique et d'...,École doctorale Sciences et Ingénierie des Sys...,Amel Borgi,"Rim Faiz, Djamal Benslimane","Ikram Amous-Ben Amor, Nabil Layaida"
1,Amélioration des techniques de la géolocalisat...,Oumaima Liouane,Toufik Bakir,Thèse de doctorat,Sciences pour l'Ingénieur,Soutenance le 27/12/2022,Bourgogne Franche-Comté en cotutelle avec Fac...,Laboratoire : Imagerie et Vision Artificielle ...,École doctorale Sciences pour l'ingénieur et m...,Kamel Ben Othman,"Abdessalem Ben Abdelali, Smain Femmam, Camel T...","Hajer Bouzaouache, Selma Boumerdassi"
2,Régularisation de problèmes inverses tensoriel...,Oumaima Benchettou,"Abderrahman Bouhamidi, Abdeslem Hafid Bentbib",Thèse de doctorat,Mathématiques et leurs applications,Soutenance le 27/12/2022,Littoral en cotutelle avec Université Cadi Ay...,Laboratoire : Laboratoire de mathématiques pur...,"École doctorale Sciences, technologie et santé...",Hassane Sadok,"Abdellah Bnouhachem, Hassan Safouhi, Otmane So...","Abdellah Bnouhachem, Hassan Safouhi, Otmane So..."
3,Algèbre multilinéaire appliquée à la restaurat...,Asmaa Khouia,"Hassane Sadok, Abdeslem Hafid Bentbib",Thèse de doctorat,Mathématiques,Soutenance le 26/12/2022,Littoral en cotutelle avec Université Cadi Ay...,Laboratoire : Laboratoire de mathématiques pur...,"École doctorale Sciences, technologie et santé...",Nour Eddine Alaa,"Hassane Sadok, Abdeslem Hafid Bentbib, Hassan ...","Hassan Safouhi, Otmane Souhar, Abdellah Bnouha..."
4,Une contribution sur l’allocation ou la prévis...,Frédy Valé Manuel Pokou,"François Benhmad, Jules Sadefo Kamdem",Thèse de doctorat,Sciences économiques,Soutenance le 23/12/2022,Université de Montpellier (2022-....),Laboratoire : Montpellier Recherche en Économie,École doctorale Economie Gestion de Montpellie...,Pascal Nguyen,"François Benhmad, Jules Sadefo Kamdem, Pascal ...","Fredj Jawadi, Duc Khuong Nguyen"


In [112]:
PhD.columns

Index(['Title', 'Author', 'Direction', 'Type', 'Discipline',
       'Date de soutenance', 'Établissement', 'Laboratoire', 'Ecole doctorale',
       'Président', 'Examinateurs', 'Rapporteurs'],
      dtype='object')

If all the values except the title have value "Not found", then we remove the row from the dataset

Seperate the names in the columns
- Author
- Title
- Direction
- Partenaire de recherche
- Date
- Examinateurs
- Rapporteurs

## Author

In [113]:
#if author not found because it is just a profile of a person then remove row
PhD = PhD[~PhD['Author'].str.startswith('Not found', na=False)]

## Title

In [114]:
#replace The end of title with Thesis.fr with blank
PhD['Title'] = PhD['Title'].str.replace(r'\s*\|\s*Theses\.fr$', '', regex=True)
#Remove row if title starts with thesis.fr because this column has no data and is useless
PhD = PhD[~PhD['Title'].str.startswith('Theses.fr', na=False)]

## Direction

In [115]:
#Direction
#Split column by commas, creating a DataFrame with multiple columns
split_direction = PhD['Direction'].str.split(',', expand=True)

#Rename the new columns to examinateur_1, examinateur_2, etc.
split_direction.columns = [f'Direction_{i+1}' for i in range(split_direction.shape[1])]

#Concatenate the original DataFrame with the split columns
PhD = pd.concat([PhD, split_direction], axis=1)

#Drop the original column
#PhD = PhD.drop(columns=['Direction'])

PhD.head()

,Title,Author,Direction,Type,Discipline,Date de soutenance,Établissement,Laboratoire,Ecole doctorale,Président,Examinateurs,Rapporteurs,Direction_1,Direction_2
0,Efficacité énergétique des phases de conceptio...,Issam Ghabri,"Ladjel Bellatreche, Sadok Ben Yahia",Thèse de doctorat,Informatique et applications,Soutenance le 29/12/2022,"Chasseneuil-du-Poitou, Ecole nationale supérie...",Laboratoire : Laboratoire d'Informatique et d'...,École doctorale Sciences et Ingénierie des Sys...,Amel Borgi,"Rim Faiz, Djamal Benslimane","Ikram Amous-Ben Amor, Nabil Layaida",Ladjel Bellatreche,Sadok Ben Yahia
1,Amélioration des techniques de la géolocalisat...,Oumaima Liouane,Toufik Bakir,Thèse de doctorat,Sciences pour l'Ingénieur,Soutenance le 27/12/2022,Bourgogne Franche-Comté en cotutelle avec Fac...,Laboratoire : Imagerie et Vision Artificielle ...,École doctorale Sciences pour l'ingénieur et m...,Kamel Ben Othman,"Abdessalem Ben Abdelali, Smain Femmam, Camel T...","Hajer Bouzaouache, Selma Boumerdassi",Toufik Bakir,None
2,Régularisation de problèmes inverses tensoriel...,Oumaima Benchettou,"Abderrahman Bouhamidi, Abdeslem Hafid Bentbib",Thèse de doctorat,Mathématiques et leurs applications,Soutenance le 27/12/2022,Littoral en cotutelle avec Université Cadi Ay...,Laboratoire : Laboratoire de mathématiques pur...,"École doctorale Sciences, technologie et santé...",Hassane Sadok,"Abdellah Bnouhachem, Hassan Safouhi, Otmane So...","Abdellah Bnouhachem, Hassan Safouhi, Otmane So...",Abderrahman Bouhamidi,Abdeslem Hafid Bentbib
3,Algèbre multilinéaire appliquée à la restaurat...,Asmaa Khouia,"Hassane Sadok, Abdeslem Hafid Bentbib",Thèse de doctorat,Mathématiques,Soutenance le 26/12/2022,Littoral en cotutelle avec Université Cadi Ay...,Laboratoire : Laboratoire de mathématiques pur...,"École doctorale Sciences, technologie et santé...",Nour Eddine Alaa,"Hassane Sadok, Abdeslem Hafid Bentbib, Hassan ...","Hassan Safouhi, Otmane Souhar, Abdellah Bnouha...",Hassane Sadok,Abdeslem Hafid Bentbib
4,Une contribution sur l’allocation ou la prévis...,Frédy Valé Manuel Pokou,"François Benhmad, Jules Sadefo Kamdem",Thèse de doctorat,Sciences économiques,Soutenance le 23/12/2022,Université de Montpellier (2022-....),Laboratoire : Montpellier Recherche en Économie,École doctorale Economie Gestion de Montpellie...,Pascal Nguyen,"François Benhmad, Jules Sadefo Kamdem, Pascal ...","Fredj Jawadi, Duc Khuong Nguyen",François Benhmad,Jules Sadefo Kamdem


## Partenaire de recherche (Laboratoire)

In [116]:
PhD['Laboratoire'] = PhD['Laboratoire'].str.replace(r'^Laboratoire :\s*', '', regex=True)

PhD.head()

,Title,Author,Direction,Type,Discipline,Date de soutenance,Établissement,Laboratoire,Ecole doctorale,Président,Examinateurs,Rapporteurs,Direction_1,Direction_2
0,Efficacité énergétique des phases de conceptio...,Issam Ghabri,"Ladjel Bellatreche, Sadok Ben Yahia",Thèse de doctorat,Informatique et applications,Soutenance le 29/12/2022,"Chasseneuil-du-Poitou, Ecole nationale supérie...",Laboratoire d'Informatique et d'Automatique po...,École doctorale Sciences et Ingénierie des Sys...,Amel Borgi,"Rim Faiz, Djamal Benslimane","Ikram Amous-Ben Amor, Nabil Layaida",Ladjel Bellatreche,Sadok Ben Yahia
1,Amélioration des techniques de la géolocalisat...,Oumaima Liouane,Toufik Bakir,Thèse de doctorat,Sciences pour l'Ingénieur,Soutenance le 27/12/2022,Bourgogne Franche-Comté en cotutelle avec Fac...,Imagerie et Vision Artificielle (ImVia) (Dijon),École doctorale Sciences pour l'ingénieur et m...,Kamel Ben Othman,"Abdessalem Ben Abdelali, Smain Femmam, Camel T...","Hajer Bouzaouache, Selma Boumerdassi",Toufik Bakir,None
2,Régularisation de problèmes inverses tensoriel...,Oumaima Benchettou,"Abderrahman Bouhamidi, Abdeslem Hafid Bentbib",Thèse de doctorat,Mathématiques et leurs applications,Soutenance le 27/12/2022,Littoral en cotutelle avec Université Cadi Ay...,Laboratoire de mathématiques pures et appliqué...,"École doctorale Sciences, technologie et santé...",Hassane Sadok,"Abdellah Bnouhachem, Hassan Safouhi, Otmane So...","Abdellah Bnouhachem, Hassan Safouhi, Otmane So...",Abderrahman Bouhamidi,Abdeslem Hafid Bentbib
3,Algèbre multilinéaire appliquée à la restaurat...,Asmaa Khouia,"Hassane Sadok, Abdeslem Hafid Bentbib",Thèse de doctorat,Mathématiques,Soutenance le 26/12/2022,Littoral en cotutelle avec Université Cadi Ay...,Laboratoire de mathématiques pures et appliqué...,"École doctorale Sciences, technologie et santé...",Nour Eddine Alaa,"Hassane Sadok, Abdeslem Hafid Bentbib, Hassan ...","Hassan Safouhi, Otmane Souhar, Abdellah Bnouha...",Hassane Sadok,Abdeslem Hafid Bentbib
4,Une contribution sur l’allocation ou la prévis...,Frédy Valé Manuel Pokou,"François Benhmad, Jules Sadefo Kamdem",Thèse de doctorat,Sciences économiques,Soutenance le 23/12/2022,Université de Montpellier (2022-....),Montpellier Recherche en Économie,École doctorale Economie Gestion de Montpellie...,Pascal Nguyen,"François Benhmad, Jules Sadefo Kamdem, Pascal ...","Fredj Jawadi, Duc Khuong Nguyen",François Benhmad,Jules Sadefo Kamdem


## Jury / Directeur 

## Examinateur 

In [117]:
#Examinateurs
#Split column by commas, creating a DataFrame with multiple columns
split_examinateurs = PhD['Examinateurs'].str.split(',', expand=True)

#Rename the new columns to examinateur_1, examinateur_2, etc.
split_examinateurs.columns = [f'Examinateur_{i+1}' for i in range(split_examinateurs.shape[1])]

#Concatenate the original DataFrame with the split columns
PhD = pd.concat([PhD, split_examinateurs], axis=1)

#Drop the original column
#PhD = PhD.drop(columns=['Examinateurs'])

PhD.head()

,Title,Author,Direction,Type,Discipline,Date de soutenance,Établissement,Laboratoire,Ecole doctorale,Président,...,Rapporteurs,Direction_1,Direction_2,Examinateur_1,Examinateur_2,Examinateur_3,Examinateur_4,Examinateur_5,Examinateur_6,Examinateur_7
0,Efficacité énergétique des phases de conceptio...,Issam Ghabri,"Ladjel Bellatreche, Sadok Ben Yahia",Thèse de doctorat,Informatique et applications,Soutenance le 29/12/2022,"Chasseneuil-du-Poitou, Ecole nationale supérie...",Laboratoire d'Informatique et d'Automatique po...,École doctorale Sciences et Ingénierie des Sys...,Amel Borgi,...,"Ikram Amous-Ben Amor, Nabil Layaida",Ladjel Bellatreche,Sadok Ben Yahia,Rim Faiz,Djamal Benslimane,None,None,None,None,None
1,Amélioration des techniques de la géolocalisat...,Oumaima Liouane,Toufik Bakir,Thèse de doctorat,Sciences pour l'Ingénieur,Soutenance le 27/12/2022,Bourgogne Franche-Comté en cotutelle avec Fac...,Imagerie et Vision Artificielle (ImVia) (Dijon),École doctorale Sciences pour l'ingénieur et m...,Kamel Ben Othman,...,"Hajer Bouzaouache, Selma Boumerdassi",Toufik Bakir,None,Abdessalem Ben Abdelali,Smain Femmam,Camel Tanougast,None,None,None,None
2,Régularisation de problèmes inverses tensoriel...,Oumaima Benchettou,"Abderrahman Bouhamidi, Abdeslem Hafid Bentbib",Thèse de doctorat,Mathématiques et leurs applications,Soutenance le 27/12/2022,Littoral en cotutelle avec Université Cadi Ay...,Laboratoire de mathématiques pures et appliqué...,"École doctorale Sciences, technologie et santé...",Hassane Sadok,...,"Abdellah Bnouhachem, Hassan Safouhi, Otmane So...",Abderrahman Bouhamidi,Abdeslem Hafid Bentbib,Abdellah Bnouhachem,Hassan Safouhi,Otmane Souhar,None,None,None,None
3,Algèbre multilinéaire appliquée à la restaurat...,Asmaa Khouia,"Hassane Sadok, Abdeslem Hafid Bentbib",Thèse de doctorat,Mathématiques,Soutenance le 26/12/2022,Littoral en cotutelle avec Université Cadi Ay...,Laboratoire de mathématiques pures et appliqué...,"École doctorale Sciences, technologie et santé...",Nour Eddine Alaa,...,"Hassan Safouhi, Otmane Souhar, Abdellah Bnouha...",Hassane Sadok,Abdeslem Hafid Bentbib,Hassane Sadok,Abdeslem Hafid Bentbib,Hassan Safouhi,Otmane Souhar,Abdellah Bnouhachem,None,None
4,Une contribution sur l’allocation ou la prévis...,Frédy Valé Manuel Pokou,"François Benhmad, Jules Sadefo Kamdem",Thèse de doctorat,Sciences économiques,Soutenance le 23/12/2022,Université de Montpellier (2022-....),Montpellier Recherche en Économie,École doctorale Economie Gestion de Montpellie...,Pascal Nguyen,...,"Fredj Jawadi, Duc Khuong Nguyen",François Benhmad,Jules Sadefo Kamdem,François Benhmad,Jules Sadefo Kamdem,Pascal Nguyen,Fredj Jawadi,Duc Khuong Nguyen,Bernard Kamsu-Foguem,None


## Rapporteurs

In [118]:
#Rapporteurs
#Split column by commas, creating a DataFrame with multiple columns
split_rapporteur = PhD['Rapporteurs'].str.split(',', expand=True)

#Rename the new columns to examinateur_1, examinateur_2, etc.
split_rapporteur.columns = [f'Rapporteur_{i+1}' for i in range(split_rapporteur.shape[1])]

#Concatenate the original DataFrame with the split columns
PhD = pd.concat([PhD, split_rapporteur], axis=1)

#Drop the original column
#PhD = PhD.drop(columns=['Rapporteurs'])

PhD.head()

,Title,Author,Direction,Type,Discipline,Date de soutenance,Établissement,Laboratoire,Ecole doctorale,Président,...,Examinateur_2,Examinateur_3,Examinateur_4,Examinateur_5,Examinateur_6,Examinateur_7,Rapporteur_1,Rapporteur_2,Rapporteur_3,Rapporteur_4
0,Efficacité énergétique des phases de conceptio...,Issam Ghabri,"Ladjel Bellatreche, Sadok Ben Yahia",Thèse de doctorat,Informatique et applications,Soutenance le 29/12/2022,"Chasseneuil-du-Poitou, Ecole nationale supérie...",Laboratoire d'Informatique et d'Automatique po...,École doctorale Sciences et Ingénierie des Sys...,Amel Borgi,...,Djamal Benslimane,None,None,None,None,None,Ikram Amous-Ben Amor,Nabil Layaida,None,None
1,Amélioration des techniques de la géolocalisat...,Oumaima Liouane,Toufik Bakir,Thèse de doctorat,Sciences pour l'Ingénieur,Soutenance le 27/12/2022,Bourgogne Franche-Comté en cotutelle avec Fac...,Imagerie et Vision Artificielle (ImVia) (Dijon),École doctorale Sciences pour l'ingénieur et m...,Kamel Ben Othman,...,Smain Femmam,Camel Tanougast,None,None,None,None,Hajer Bouzaouache,Selma Boumerdassi,None,None
2,Régularisation de problèmes inverses tensoriel...,Oumaima Benchettou,"Abderrahman Bouhamidi, Abdeslem Hafid Bentbib",Thèse de doctorat,Mathématiques et leurs applications,Soutenance le 27/12/2022,Littoral en cotutelle avec Université Cadi Ay...,Laboratoire de mathématiques pures et appliqué...,"École doctorale Sciences, technologie et santé...",Hassane Sadok,...,Hassan Safouhi,Otmane Souhar,None,None,None,None,Abdellah Bnouhachem,Hassan Safouhi,Otmane Souhar,None
3,Algèbre multilinéaire appliquée à la restaurat...,Asmaa Khouia,"Hassane Sadok, Abdeslem Hafid Bentbib",Thèse de doctorat,Mathématiques,Soutenance le 26/12/2022,Littoral en cotutelle avec Université Cadi Ay...,Laboratoire de mathématiques pures et appliqué...,"École doctorale Sciences, technologie et santé...",Nour Eddine Alaa,...,Abdeslem Hafid Bentbib,Hassan Safouhi,Otmane Souhar,Abdellah Bnouhachem,None,None,Hassan Safouhi,Otmane Souhar,Abdellah Bnouhachem,None
4,Une contribution sur l’allocation ou la prévis...,Frédy Valé Manuel Pokou,"François Benhmad, Jules Sadefo Kamdem",Thèse de doctorat,Sciences économiques,Soutenance le 23/12/2022,Université de Montpellier (2022-....),Montpellier Recherche en Économie,École doctorale Economie Gestion de Montpellie...,Pascal Nguyen,...,Jules Sadefo Kamdem,Pascal Nguyen,Fredj Jawadi,Duc Khuong Nguyen,Bernard Kamsu-Foguem,None,Fredj Jawadi,Duc Khuong Nguyen,None,None


In [119]:
PhD.columns

Index(['Title', 'Author', 'Direction', 'Type', 'Discipline',
       'Date de soutenance', 'Établissement', 'Laboratoire', 'Ecole doctorale',
       'Président', 'Examinateurs', 'Rapporteurs', 'Direction_1',
       'Direction_2', 'Examinateur_1', 'Examinateur_2', 'Examinateur_3',
       'Examinateur_4', 'Examinateur_5', 'Examinateur_6', 'Examinateur_7',
       'Rapporteur_1', 'Rapporteur_2', 'Rapporteur_3', 'Rapporteur_4'],
      dtype='object')

## Fix the date

In [120]:
# Remove the soutenance string in front of the date
PhD['Date de soutenance'] = PhD['Date de soutenance'].str.replace(r'^Soutenance le\s*', '', case=False, regex=True)

# Ensure that the remainer is a date 
# ( Errors='coerce' will handle invalid date formats by returning NaT for those entries )
PhD['Date de soutenance'] = pd.to_datetime(PhD['Date de soutenance'], errors='coerce', dayfirst=True)

PhD.head()

,Title,Author,Direction,Type,Discipline,Date de soutenance,Établissement,Laboratoire,Ecole doctorale,Président,...,Examinateur_2,Examinateur_3,Examinateur_4,Examinateur_5,Examinateur_6,Examinateur_7,Rapporteur_1,Rapporteur_2,Rapporteur_3,Rapporteur_4
0,Efficacité énergétique des phases de conceptio...,Issam Ghabri,"Ladjel Bellatreche, Sadok Ben Yahia",Thèse de doctorat,Informatique et applications,2022-12-29,"Chasseneuil-du-Poitou, Ecole nationale supérie...",Laboratoire d'Informatique et d'Automatique po...,École doctorale Sciences et Ingénierie des Sys...,Amel Borgi,...,Djamal Benslimane,None,None,None,None,None,Ikram Amous-Ben Amor,Nabil Layaida,None,None
1,Amélioration des techniques de la géolocalisat...,Oumaima Liouane,Toufik Bakir,Thèse de doctorat,Sciences pour l'Ingénieur,2022-12-27,Bourgogne Franche-Comté en cotutelle avec Fac...,Imagerie et Vision Artificielle (ImVia) (Dijon),École doctorale Sciences pour l'ingénieur et m...,Kamel Ben Othman,...,Smain Femmam,Camel Tanougast,None,None,None,None,Hajer Bouzaouache,Selma Boumerdassi,None,None
2,Régularisation de problèmes inverses tensoriel...,Oumaima Benchettou,"Abderrahman Bouhamidi, Abdeslem Hafid Bentbib",Thèse de doctorat,Mathématiques et leurs applications,2022-12-27,Littoral en cotutelle avec Université Cadi Ay...,Laboratoire de mathématiques pures et appliqué...,"École doctorale Sciences, technologie et santé...",Hassane Sadok,...,Hassan Safouhi,Otmane Souhar,None,None,None,None,Abdellah Bnouhachem,Hassan Safouhi,Otmane Souhar,None
3,Algèbre multilinéaire appliquée à la restaurat...,Asmaa Khouia,"Hassane Sadok, Abdeslem Hafid Bentbib",Thèse de doctorat,Mathématiques,2022-12-26,Littoral en cotutelle avec Université Cadi Ay...,Laboratoire de mathématiques pures et appliqué...,"École doctorale Sciences, technologie et santé...",Nour Eddine Alaa,...,Abdeslem Hafid Bentbib,Hassan Safouhi,Otmane Souhar,Abdellah Bnouhachem,None,None,Hassan Safouhi,Otmane Souhar,Abdellah Bnouhachem,None
4,Une contribution sur l’allocation ou la prévis...,Frédy Valé Manuel Pokou,"François Benhmad, Jules Sadefo Kamdem",Thèse de doctorat,Sciences économiques,2022-12-23,Université de Montpellier (2022-....),Montpellier Recherche en Économie,École doctorale Economie Gestion de Montpellie...,Pascal Nguyen,...,Jules Sadefo Kamdem,Pascal Nguyen,Fredj Jawadi,Duc Khuong Nguyen,Bernard Kamsu-Foguem,None,Fredj Jawadi,Duc Khuong Nguyen,None,None


Export the data into csv

In [121]:
PhD.shape

(10, 25)

In [122]:
PhD['Title'] = PhD['Title'].str.replace(r'\s*\|\s*Theses\.fr$', '', regex=True)

In [123]:
PhD = PhD[~PhD['Title'].str.startswith('Theses.fr', na=False)]

In [124]:
PhD.to_csv('Alex_Kiki_Yudita_PhD_2022.csv', index=False, encoding='utf-8-sig')